<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork899-2023-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# Peer Review Assignment - Data Engineer - ETL


Estimated time needed: **20** minutes


## Objectives

In this final part you will:

- Run the ETL process
- Extract bank and market cap data from the JSON file `bank_market_cap.json`
- Transform the market cap currency using the exchange rate data
- Load the transformed data into a seperate CSV


For this lab, we are going to be using Python and several Python libraries. Some of these libraries might be installed in your lab environment or in SN Labs. Others may need to be installed by you. The cells below will install these libraries when executed.


In [ ]:
#!mamba install pandas==1.3.3 -y
#!mamba install requests==2.26.0 -y

## Imports

Import any additional libraries you may need here.


In [1]:
import glob
import pandas as pd
from datetime import datetime

As the exchange rate fluctuates, we will download the same dataset to make marking simpler. This will be in the same format as the dataset you used in the last section  


In [2]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv

--2023-08-30 22:25:14--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2815 (2.7K) [application/json]
Saving to: ‘bank_market_cap_1.json’

bank_market_cap_1.j 100%[===================>]   2.75K  --.-KB/s    in 0s      

2023-08-30 22:25:14 (28.0 MB/s) - ‘bank_market_cap_1.json’ saved [2815/2815]

--2023-08-30 22:25:15--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Tra

## Extract


### JSON Extract Function

This function will extract JSON files.


In [3]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe

## Extract Function

Define the extract function that finds JSON file `bank_market_cap_1.json` and calls the function created above to extract data from them. Store the data in a `pandas` dataframe. Use the following list for the columns.


In [4]:
columns=['Name','Market Cap (US$ Billion)']

In [6]:
def extract():
    # Write your code here
    file = 'bank_market_cap_1.json'
    df1 = extract_from_json(file)
    df2 = df1[columns]
    return df2

In [8]:
df = extract()
df

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399
...,...,...
65,Ping An Bank,37.993
66,Standard Chartered,37.319
67,United Overseas Bank,35.128
68,QNB Group,33.560


<b>Question 1</b> Load the file <code>exchange_rates.csv</code> as a dataframe and find the exchange rate for British pounds with the symbol <code>GBP</code>, store it in the variable  <code>exchange_rate</code>, you will be asked for the number. Hint: set the parameter  <code>index_col</code> to 0.


In [11]:
df3 = pd.read_csv('exchange_rates.csv',index_col=0)
df3.head()

,Rates
AUD,1.297088
BGN,1.608653
BRL,5.409196
CAD,1.271426
CHF,0.886083


In [13]:
# Write your code here
exchange_rate = df3.loc['GBP','Rates']
print('Exchange Rate: ', exchange_rate)

Exchange Rate:  0.7323984208000001


## Transform

Using <code>exchange_rate</code> and the `exchange_rates.csv` file find the exchange rate of USD to GBP. Write a transform function that

1. Changes the `Market Cap (US$ Billion)` column from USD to GBP
2. Rounds the Market Cap (US$ Billion)` column to 3 decimal places
3. Rename `Market Cap (US$ Billion)` to `Market Cap (GBP$ Billion)`


In [15]:
def transform(dataFrame, exchange_rate):
    usdtogbp = dataFrame.loc["USD", "Rates"] / exchange_rate
    df_usd = extract()
    df_usd.loc[: , "Market Cap (GBP$ Billion)"] = round(df_usd.iloc[: ,1] / usdtogbp, 3)
    df_gbp = df_usd.drop("Market Cap (US$ Billion)", axis=1)
    return df_gbp, usdtogbp

In [16]:
df_gbp, usdtogbp = transform(df3, exchange_rate)
print("Exchange Rate from USD to GBP: ", round(usdtogbp,3))
print(df_gbp.head())

Exchange Rate from USD to GBP:  1.365
                                      Name  Market Cap (GBP$ Billion)
0                           JPMorgan Chase                    286.319
1  Industrial and Commercial Bank of China                    252.834
2                          Bank of America                    238.272
3                              Wells Fargo                    225.588
4                  China Construction Bank                    188.519


## Load

Create a function that takes a dataframe and load it to a csv named `bank_market_cap_gbp.csv`. Make sure to set `index` to `False`.


In [31]:
def load(dataFrame, filename):
    # Write your code here
    df_gbp.to_csv(filename, index=False)
    

## Logging Function


Write the logging function <code>log</code> to log your data:


In [18]:
def log(message):
    # Write your code here
    with open('log.txt', "a") as f:
        dt = datetime.today().strftime("%Y-%m-%d %H:%M-%S")
        f.write(str(dt) + "   " + str(message) + "\n")
    

## Running the ETL Process


Log the process accordingly using the following <code>"ETL Job Started"</code> and <code>"Extract phase Started"</code>


In [20]:
# Write your code here
log("ETL Job Started")
log("Extract Phase Started")

### Extract


<code>Question 2</code> Use the function <code>extract</code>, and print the first 5 rows, take a screen shot:


In [21]:
# Call the function here
df0 = extract()
# Print the rows here
df0.head()

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399


Log the data as <code>"Extract phase Ended"</code>


In [22]:
# Write your code here
log("Extract Phase Ended")

### Transform


Log the following  <code>"Transform phase Started"</code>


In [23]:
# Write your code here
log("Transform Phase Started")

<code>Question 3</code> Use the function <code>transform</code> and print the first 5 rows of the output, take a screen shot:


In [24]:
# Call the function here
df_gbp, usdtogbp = transform(df3, exchange_rate)
# Print the first 5 rows here
df_gbp.head()

,Name,Market Cap (GBP$ Billion)
0,JPMorgan Chase,286.319
1,Industrial and Commercial Bank of China,252.834
2,Bank of America,238.272
3,Wells Fargo,225.588
4,China Construction Bank,188.519


Log your data <code>"Transform phase Ended"</code>


In [25]:
# Write your code here
log("Transform Phase Ended")

### Load


Log the following `"Load phase Started"`.


In [26]:
# Write your code here
log("Load Phase Started")

Call the load function


In [32]:
# Write your code here
load(df_gbp, "bank_market_cap_gbp.csv")

Log the following `"Load phase Ended"`.


In [33]:
# Write your code here
log("Load Phase Ended")

In [34]:
!cat log.txt

2023-08-30 23:02-10   ETL Job Started
2023-08-30 23:02-10   Extract Phase Started
2023-08-30 23:02-37   Extract Phase Ended
2023-08-30 23:02-50   Transform Phase Started
2023-08-30 23:04-01   Transform Phase Ended
2023-08-30 23:04-15   Load Phase Started
2023-08-30 23:09-18   Load Phase Ended


## Authors


Ramesh Sannareddy, Joseph Santrcangelo and Azim Hirjani


### Other Contributors


Rav Ahuja


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By        | Change Description                 |
| ----------------- | ------- | ----------------- | ---------------------------------- |
| 2020-11-25        | 0.1     | Ramesh Sannareddy | Created initial version of the lab |


 Copyright © 2020 IBM Corporation. This notebook and its source code are released under the terms of the [MIT License](https://cognitiveclass.ai/mit-license?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork899-2023-01-01&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBM-DA0321EN-SkillsNetwork-21426264&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ).
